![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_DistilBertForMultipleChoice.ipynb)

# Import OpenVINO DistilBertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting DistilBertForMultipleChoice  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for DistilBertForMultipleChoice from DistilBertForMultipleChoice  and they have to be in `Multiple Choice` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install --upgrade pip
!pip install -q --upgrade transformers[onnx] optimum-intel openvino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 182.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 197.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 184.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- Normally, to load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. However, ForMultipleChoice is not yet available so we will use `openvino.convert_model()` after exporting ONNX model
- We'll use [irfanamal/bert_multiple_choice](https://huggingface.co/irfanamal/bert_multiple_choice) model from HuggingFace as an example
- We also need the `vocab.txt` saved from `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "daze-unlv/distilbert-distilbert-base-uncased"
ONNX_MODEL_PATH = f"onnx_models/distilbert_multiple_choice"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
!ls -lh {ONNX_MODEL_PATH}

total 257M
-rw-r--r-- 1 root root  512 May  4 18:13 config.json
-rw-r--r-- 1 root root 256M May  4 18:13 model.onnx
-rw-r--r-- 1 root root  695 May  4 18:13 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K May  4 18:13 tokenizer_config.json
-rw-r--r-- 1 root root 695K May  4 18:13 tokenizer.json
-rw-r--r-- 1 root root 227K May  4 18:13 vocab.txt


In [4]:
import openvino as ov

# ov.convert_model returns an openvino.runtime.Model object
ov_model = ov.convert_model(f"{ONNX_MODEL_PATH}/model.onnx")

OPENVINO_MODEL_PATH = "./openvino_models"
# then model can be serialized to *.xml & *.bin files
ov.save_model(ov_model, f"{OPENVINO_MODEL_PATH}/openvino_model.xml")

In [5]:
!ls -lh {OPENVINO_MODEL_PATH}

total 129M
-rw-r--r-- 1 root root 128M May  4 18:13 openvino_model.bin
-rw-r--r-- 1 root root 310K May  4 18:13 openvino_model.xml


In [6]:
!mkdir {OPENVINO_MODEL_PATH}/assets

In [7]:
!mv {ONNX_MODEL_PATH}/vocab.txt {OPENVINO_MODEL_PATH}/assets

Let's have a look inside these two directories and see what we are dealing with:

In [8]:
!ls -lR {OPENVINO_MODEL_PATH}

./openvino_models:
total 131096
drwxr-xr-x 2 root root      4096 May  4 18:13 assets
-rw-r--r-- 1 root root 133912792 May  4 18:13 openvino_model.bin
-rw-r--r-- 1 root root    317207 May  4 18:13 openvino_model.xml

./openvino_models/assets:
total 228
-rw-r--r-- 1 root root 231508 May  4 18:13 vocab.txt


## Import and Save DistilBertForMultipleChoice in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [9]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/spark-nlp-assembly-6.0.0.jar /content/
!cp /content/drive/MyDrive/spark_nlp-6.0.0-py2.py3-none-any.whl /content/

!pip install /content/spark_nlp-6.0.0-py2.py3-none-any.whl
import sparknlp
from pyspark.sql import SparkSession

# let's start Spark with Spark NLP
spark = SparkSession.builder \
			.master("local[*]") \
			.config("spark.jars", "/content/spark-nlp-assembly-6.0.0.jar") \
			.config("spark.driver.memory", "64g") \
			.config("spark.driver.maxResultSize", "0G") \
			.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
			.config("spark.kryoserializer.buffer.max", "2000M") \
			.config('spark.executor.logs.rolling.strategy', 'time') \
			.config('spark.executor.logs.rolling.maxRetainedFiles', '10') \
			.getOrCreate()

print("Apache Spark version: {}".format(spark.version))

Mounted at /content/drive
Processing ./spark_nlp-6.0.0-py2.py3-none-any.whl
Apache Spark version: 3.5.5


Let's start Spark with Spark NLP included via our simple `start()` function

- Let's use `loadSavedModel` functon in `DistilBertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForMultipleChoice` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [10]:
from sparknlp.annotator import *
from sparknlp.base import *


distilBertMultpleChoiceClassifier = DistilBertForMultipleChoice.loadSavedModel(
     f"{OPENVINO_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
distilBertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {ONNX_MODEL_PATH}
!rm -rf {OPENVINO_MODEL_PATH}

Awesome 😎  !

This is your`DistilBertForMultipleChoice` model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 131116
-rw-r--r-- 1 root root 134250762 May  4 18:14 distilbert_mc_classification_openvino
drwxr-xr-x 3 root root      4096 May  4 18:14 fields
drwxr-xr-x 2 root root      4096 May  4 18:14 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `DistilBertForMultipleChoice` model 😊

In [14]:
 testing_data = [
            ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
             "It is eaten with a fork and a knife, It is eaten while held in the hand."),

            ("The Eiffel Tower is located in which country?",
             "Germany, France, Italy"),

            ("Which animal is known as the king of the jungle?",
             "Lion, Elephant, Tiger, Leopard"),

            ("Water boils at what temperature?",
             "90°C, 120°C, 100°C"),

            ("Which planet is known as the Red Planet?",
             "Jupiter, Mars, Venus"),

            ("Which language is primarily spoken in Brazil?",
             "Spanish, Portuguese, English"),

            ("The Great Wall of China was built to protect against invasions from which group?",
             "The Greeks, The Romans, The Mongols, The Persians"),

            ("Which chemical element has the symbol 'O'?",
             "Oxygenm, Osmium, Ozone"),

            ("Which continent is the Sahara Desert located in?",
             "Asia, Africa, South America"),

            ("Which artist painted the Mona Lisa?",
             "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
        ]

In [15]:
testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = MultiDocumentAssembler() \
            .setInputCols(["question", "choices"]) \
            .setOutputCols(["document_question", "document_choices"])

distilbert_for_multiple_choice = DistilBertForMultipleChoice() \
  .load(f"{MODEL_NAME}_spark_nlp_openvino") \
  .setInputCols(["document_question", "document_choices"])\
  .setOutputCol("answer") \
  .setBatchSize(4)

pipeline = Pipeline(stages=[document_assembler, distilbert_for_multiple_choice])
pipeline_model = pipeline.fit(testing_df)

pipeline_df = pipeline_model.transform(testing_df)

pipeline_df.select("answer").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+
|answer                                                                                                     |
+-----------------------------------------------------------------------------------------------------------+
|[{chunk, 0, 34, It is eaten with a fork and a knife, {sentence -> 0, chunk -> 0, score -> 0.60487646}, []}]|
|[{chunk, 0, 6, Germany, {sentence -> 0, chunk -> 0, score -> 0.39134774}, []}]                             |
|[{chunk, 0, 5,  Tiger, {sentence -> 0, chunk -> 0, score -> 0.28978875}, []}]                              |
|[{chunk, 0, 3, 90°C, {sentence -> 0, chunk -> 0, score -> 0.35916173}, []}]                                |
|[{chunk, 0, 6, Jupiter, {sentence -> 0, chunk -> 0, score -> 0.35947335}, []}]                             |
|[{chunk, 0, 7,  English, {sentence -> 0, chunk -> 0, score -> 0.36399665}, []}]                            |
|[{chunk, 

That's it! You can now go wild and use hundreds of `DistilBertForMultipleChoice` models from HuggingFace 🤗 in Spark NLP 🚀
